In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import json
from elasticsearch import Elasticsearch, helpers
import configparser
import pandas as pd
from datetime import datetime, timedelta

present_date = str(datetime.utcnow() + timedelta(hours=9))[:10]

index="sm_best_100_"

config = configparser.ConfigParser()
config.read('example.ini')

es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

if es.indices.exists(index = index + present_date):
    pass
else:
    es.indices.create(index = index + present_date, body = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "nori": {
                        "tokenizer": "nori_tokenizer"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "sm_subject": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "genre": {
                    "type": "keyword"
                },
                "sm_writer": {
                    "type": "text",
                    "analyzer": "nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "sm_pulisher": {
                    "type": "text",
                    "analyzer":"nori",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "sm_date": {
                    "type": "text",
                    "fields":{
                        "keyword":{
                            "type":"keyword"
                        }
                    }
                },
                "sm_count": {
                    "type": "keyword"
                },
                "sm_reservation": {
                    "type": "integer"
                },
                "sm_checkout": {
                    "type": "integer"
                },
                "sm_like": {
                    "type": "integer"
                },
                "present_date": {
                    "type": "date"
                }
            }
        }
    })

# 상명대학교 전자도서관 베스트 셀러 URL
URL = "https://libebook.smu.ac.kr/FxLibrary/product/list/?category_type=book&cateopt=best" 

# 총 페이지 수 추출 함수
def get_page_count():
    result = requests.get(URL)

    soup = BeautifulSoup(result.text, "html.parser")

    count = len(soup.select("div.paging span a"))+1
    
    return count

# 각 페이지마다 책 정보 추출 함수
def extract_books(page_count):
    book_list = []
    
    for page in range(page_count):
        print(f"Scrapping page {page+1}")
        
        rs = requests.get(f"https://libebook.smu.ac.kr/FxLibrary/product/list/?itemdv=1&sort=3&page={page+1}&itemCount=20&pageCount=10&category=&middlecategory=&cateopt=best&group_num=recommand&catenavi=main&category_type=book&searchoption=&keyoption=&keyoption2=&keyword=&listfilter=all_list&selectview=list_on&searchType=&name=&publisher=&author=&terminal=")
        
        soup = BeautifulSoup(rs.text, "html.parser")
        
        results=soup.select("div.book_list ul.list.typelist li.item")
        
        for rs in results:
            try:
                subject = rs.select_one("div.subject a").text
                
                category = rs.select_one("div.category span").text
    
                writer = rs.select("div.info ul.i1 a")[0].text

                publisher = rs.select("div.info ul.i1 a")[1].text

                date = rs.select("div.info ul.i1 li")[2].text
    
                if date == '0000-00-00' or date == "NaN":         
                    date = '1970-01-01'
                
                count = rs.select("div.info ul.i2 li")[0].text
                count = count.split(' ')[1]
                
                reservation = rs.select("div.info ul.i2 li")[1].text
                reservation = reservation.split(' ')[1]

                checkout = rs.select("div.info ul.i2 li")[2].text
                checkout = checkout.split(' ')[1]
                
                like = rs.select("div.info ul.i2 li")[3].text
                like = like.split(' ')[1]
            
                book_list.append([subject, category, writer, publisher, date, count, reservation, checkout, like, present_date])
            
                es.index(
                    index = index + present_date,
                    document = {
                        'sm_subject':subject,
                        'genre':category,
                        'sm_writer':writer,
                        'sm_publisher':publisher,
                        'sm_date':date,
                        'sm_count':count,
                        'sm_reservation':reservation,
                        'sm_checkout':checkout,
                        'sm_like':like,
                        'present_date':present_date
                    })
                
            except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
                print('예외가 발생했습니다.', e)
                pass
            
    return book_list

# CSV 저장
page_count = get_page_count()

books = extract_books(page_count)

df = pd.DataFrame(books, columns = ["subject", "genre", "writer", "pulisher", "date", "count", "reservation", "checkout","like", "present_date"])

df.to_csv(f"{index + present_date}.csv", index = False, encoding = 'utf-8-sig')

C:\Users\shjo\AppData\Local\Temp\ipykernel_22280\1144077095.py:17: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(
C:\Users\shjo\AppData\Local\Temp\ipykernel_22280\1144077095.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index = index + present_date, body = {


Scrapping page 1
Scrapping page 2
Scrapping page 3
Scrapping page 4
Scrapping page 5


In [8]:
df

,subject,genre,writer,pulisher,date,count,reservation,checkout,like,present_date
0,트렌드 코리아 2023,[경제/비즈니스],"김난도, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이향은, 한다혜, 이혜원, 추예린",미래의창,2022-10-06,20/20,6,164,1,2023-02-24
1,물고기는 존재하지 않는다,[자연/과학],룰루 밀러,곰출판,2022-01-19,20/20,2,194,2,2023-02-24
2,지구 끝의 온실,[문학],김초엽,자이언트북스,2022-04-21,10/10,1,139,0,2023-02-24
3,불편한 편의점,[문학],김호연,나무옆의자,2022-02-22,8/10,0,155,0,2023-02-24
4,달러구트 꿈 백화점,[문학],이미예,북닻,2020-07-28,10/10,0,204,4,2023-02-24
...,...,...,...,...,...,...,...,...,...,...
95,핸즈온 머신러닝 (제2판),[강의지원도서],오렐리앙 제롱,한빛미디어,2020-06-03,2/3,0,3,0,2023-02-24
96,168시간 일주일 사용법,[경제/비즈니스],케빈 호건,비전코리아,2011-03-10,0/3,0,543,0,2023-02-24
97,"1일 1페이지, 세상에서 가장 짧은 교양 수업 365 : 현대문화편",[인문],"데이비드 S. 키더, 노아 D. 오펜하임",위즈덤하우스,2020-12-14,1/2,0,49,0,2023-02-24
98,3년 후 부의 흐름이 보이는 경제지표 정독법,[경제/비즈니스],김영익,한즈미디어,2022-08-29,1/2,0,2,0,2023-02-24
